In [1]:
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

In [2]:
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)
utility.drop_collection("partition_key_collection")

In [3]:
## Field Schema

song_name = FieldSchema(
  name="song_name",
  dtype=DataType.VARCHAR,
  max_length=200,
)
song_id = FieldSchema(
  name="song_id",
  dtype=DataType.INT64,
  is_primary=True,
)
listen_count = FieldSchema(
  name="listen_count",
  dtype=DataType.INT64,
)
song_vec = FieldSchema(
  name="song_vec",
  dtype=DataType.FLOAT_VECTOR,
  dim=2
)
language = FieldSchema(
  name="language",
  dtype=DataType.VARCHAR,
  max_length=64,
  is_partition_key=True
)

# Collection schema
collection_schema = CollectionSchema(
  fields=[song_name, song_id, listen_count, song_vec, language],
  description="Album Songs"
)

# Create collection

collection = Collection(
    name="partition_key_collection",
    schema=collection_schema,
    partition_key_field="language",
    using='default')

utility.list_collections()

['Album1', 'dynamic_schema_example', 'partition_key_collection']

In [ ]:
## Vector Similarity Search

results = collection.search(
	data=[[0.1, 0.2]], 
	anns_field="song_vec", 
	param={"metric_type": "L2", "params": {"search_k": 64}},
	limit=5, 
	expr='language=="english"', # Limit the search for a partition key
	output_fields=['song_name']
)

for result in results[0]:
    print (result)